In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tabgan

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.8 MB/s eta 0:00:00


In [ ]:
from tabgan.sampler import OriginalGenerator, GANGenerator
import pandas as pd
import numpy as np
# train_data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/플젝테스트/제공_모델_코드/기초데이터/train_data.csv',encoding='euc-kr',index_col=0)
# test_data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/플젝테스트/제공_모델_코드/기초데이터/test_data.csv',encoding='euc-kr',index_col=0)


# 파일에서 데이터 불러오기
train_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/플젝테스트/제공_모델_코드/생성모델/제공_모델_코드 2/기초데이터/train_data.csv',encoding='euc-kr',index_col=0)
test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/플젝테스트/제공_모델_코드/생성모델/제공_모델_코드 2/기초데이터/train_data.csv',encoding='euc-kr',index_col=0)
# 기존 인덱스를 열로 변환하고 새로운 정수 인덱스 설정

test_data.drop(['index'],inplace=True,axis=1)
train_data.drop(['index'],inplace=True,axis=1)
# # 필요한 경우, 특성과 타겟을 분리
# train_features = train_data.drop(['색상_L*', '색상_a*', '색상_b*'], axis=1)
# train_targets = train_data[['색상_L*', '색상_a*', '색상_b*']]

# test_features = test_data.drop(['색상_L*', '색상_a*', '색상_b*'], axis=1)
# # 테스트 타겟이 필요한 경우
# # test_targets = test_data[['L', 'A', 'B']]


In [ ]:
train_data

,소재_경사정보_V1,소재_경사정보_V2,소재_경사정보_V3,소재_경사정보_V4,소재_경사정보_V5,소재_경사정보_V6,소재_경사정보_V7,소재_경사정보_V8,소재_경사정보_V9,소재_경사정보_V10,...,배합_염료3_Black_prop,배합_염료1_Grey_prop,배합_염료2_Grey_prop,배합_염료3_Grey_prop,배합_염료2_Brown_prop,배합_염료1_Brown_prop,배합_염료3_Brown_prop,색상_L*,색상_a*,색상_b*
1748,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0,0,0,0,0,0,0,20.7300,8.2000,-20.2100
3730,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,96,Filaments,150,150,0,...,0,0,0,0,0,0,0,72.4890,25.1515,84.0940
308,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0,0,0,0,0,0,0,50.1900,6.8500,-20.2500
930,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0,0,0,0,0,0,0,50.6250,62.5350,17.1050
49,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0,0,0,0,1,0,0,57.1100,34.0900,55.9900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0,0,0,0,0,0,0,71.6800,-8.5400,-17.1050
3286,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,96,Filaments,150,150,0,...,0,0,0,0,0,0,0,35.5640,-6.9755,-30.2565
1669,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0,0,0,0,0,0,0,44.6555,21.8540,-13.9980
2625,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,96,Filaments,150,150,0,...,0,0,0,0,0,0,0,54.3150,20.6165,-12.6340


In [ ]:
cate_feature_col=['소재_경사정보_V1',
 '소재_경사정보_V4',
 '소재_경사정보_V5',
 '소재_경사정보_V7',
 '소재_경사정보_V16',
 '소재_위사정보_V1',
 '소재_위사정보_V4',
 '소재_위사정보_V5',
 '소재_위사정보_V7',
 '소재_위사정보_V16',
 '소재_제직정보_V1',
 '소재_제직정보_V2'
                 ]

In [ ]:


from sklearn.preprocessing import LabelEncoder


In [ ]:
categorical_names = {}
for feature in cate_feature_col:
    le = LabelEncoder()

    # 학습 데이터셋을 기준으로 LabelEncoder를 훈련시킴
    le.fit(train_data[feature])

    # 학습 및 테스트 데이터셋에 인코딩 적용
    train_data[feature] = le.transform(train_data[feature])
    test_data[feature] = le.transform(test_data[feature])

    # 인코딩된 범주 정보 저장
    categorical_names[feature] = le.classes_


In [ ]:
categorical_names

{'소재_경사정보_V1': array(['PET 재생 섬유'], dtype=object),
 '소재_경사정보_V4': array(['Semi Dull'], dtype=object),
 '소재_경사정보_V5': array(['Draw Textured Yarn'], dtype=object),
 '소재_경사정보_V7': array(['Filaments'], dtype=object),
 '소재_경사정보_V16': array(['Interlace'], dtype=object),
 '소재_위사정보_V1': array(['PET 재생 섬유'], dtype=object),
 '소재_위사정보_V4': array(['Semi Dull'], dtype=object),
 '소재_위사정보_V5': array(['Draw Textured Yarn'], dtype=object),
 '소재_위사정보_V7': array(['Filaments'], dtype=object),
 '소재_위사정보_V16': array(['Interlace'], dtype=object),
 '소재_제직정보_V1': array(['WOVEN'], dtype=object),
 '소재_제직정보_V2': array(['TWILL'], dtype=object)}

In [ ]:
categorical_names

{'소재_경사정보_V1': array(['PET 재생 섬유'], dtype=object),
 '소재_경사정보_V4': array(['Semi Dull'], dtype=object),
 '소재_경사정보_V5': array(['Draw Textured Yarn'], dtype=object),
 '소재_경사정보_V7': array(['Filaments'], dtype=object),
 '소재_경사정보_V16': array(['Interlace'], dtype=object),
 '소재_위사정보_V1': array(['PET 재생 섬유'], dtype=object),
 '소재_위사정보_V4': array(['Semi Dull'], dtype=object),
 '소재_위사정보_V5': array(['Draw Textured Yarn'], dtype=object),
 '소재_위사정보_V7': array(['Filaments'], dtype=object),
 '소재_위사정보_V16': array(['Interlace'], dtype=object),
 '소재_제직정보_V1': array(['WOVEN'], dtype=object),
 '소재_제직정보_V2': array(['TWILL'], dtype=object)}

In [ ]:
for feature in cate_feature_col:
    print(train_data[feature ].head())
    print(test_data[feature ].head())


1748    0
3730    0
308     0
930     0
49      0
Name: 소재_경사정보_V1, dtype: int64
1748    0
3730    0
308     0
930     0
49      0
Name: 소재_경사정보_V1, dtype: int64
1748    0
3730    0
308     0
930     0
49      0
Name: 소재_경사정보_V4, dtype: int64
1748    0
3730    0
308     0
930     0
49      0
Name: 소재_경사정보_V4, dtype: int64
1748    0
3730    0
308     0
930     0
49      0
Name: 소재_경사정보_V5, dtype: int64
1748    0
3730    0
308     0
930     0
49      0
Name: 소재_경사정보_V5, dtype: int64
1748    0
3730    0
308     0
930     0
49      0
Name: 소재_경사정보_V7, dtype: int64
1748    0
3730    0
308     0
930     0
49      0
Name: 소재_경사정보_V7, dtype: int64
1748    0
3730    0
308     0
930     0
49      0
Name: 소재_경사정보_V16, dtype: int64
1748    0
3730    0
308     0
930     0
49      0
Name: 소재_경사정보_V16, dtype: int64
1748    0
3730    0
308     0
930     0
49      0
Name: 소재_위사정보_V1, dtype: int64
1748    0
3730    0
308     0
930     0
49      0
Name: 소재_위사정보_V1, dtype: int64
1748    0
3730    0
308   

In [ ]:

# 필요한 경우, 특성과 타겟을 분리
# 원본 범주형 열 대신 인코딩된 열을 사용
train_features = train_data.drop(['색상_L*', '색상_a*', '색상_b*'] , axis=1)
train_targets = train_data[['색상_L*', '색상_a*', '색상_b*']]

test_features = test_data.drop(['색상_L*', '색상_a*', '색상_b*'], axis=1)


In [ ]:
train_targets

,색상_L*,색상_a*,색상_b*
1748,20.7300,8.2000,-20.2100
3730,72.4890,25.1515,84.0940
308,50.1900,6.8500,-20.2500
930,50.6250,62.5350,17.1050
49,57.1100,34.0900,55.9900
...,...,...,...
839,71.6800,-8.5400,-17.1050
3286,35.5640,-6.9755,-30.2565
1669,44.6555,21.8540,-13.9980
2625,54.3150,20.6165,-12.6340


In [ ]:
test_features

,소재_경사정보_V1,소재_경사정보_V2,소재_경사정보_V3,소재_경사정보_V4,소재_경사정보_V5,소재_경사정보_V6,소재_경사정보_V7,소재_경사정보_V8,소재_경사정보_V9,소재_경사정보_V10,...,배합_염료3_Blue_prop,배합_염료1_Black_prop,배합_염료2_Black_prop,배합_염료3_Black_prop,배합_염료1_Grey_prop,배합_염료2_Grey_prop,배합_염료3_Grey_prop,배합_염료2_Brown_prop,배합_염료1_Brown_prop,배합_염료3_Brown_prop
1748,0,300,96,0,0,88,0,150,150,0,...,0.00000,0,0,0,0,0,0,0,0,0
3730,0,300,96,0,0,96,0,150,150,0,...,0.00000,0,0,0,0,0,0,0,0,0
308,0,300,96,0,0,88,0,150,150,0,...,0.00000,0,0,0,0,0,0,0,0,0
930,0,300,96,0,0,88,0,150,150,0,...,0.00000,0,0,0,0,0,0,0,0,0
49,0,300,96,0,0,88,0,150,150,0,...,0.00000,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839,0,300,96,0,0,88,0,150,150,0,...,0.00000,0,0,0,0,0,0,0,0,0
3286,0,300,96,0,0,96,0,150,150,0,...,0.00000,0,0,0,0,0,0,0,0,0
1669,0,300,96,0,0,88,0,150,150,0,...,0.00000,0,0,0,0,0,0,0,0,0
2625,0,300,96,0,0,96,0,150,150,0,...,0.33400,0,0,0,0,0,0,0,0,0


In [ ]:
new_trains = []
new_targets = []
# GANGenerator 인스턴스 생성
gan_gen = GANGenerator()
for target_col in ['색상_L*', '색상_a*', '색상_b*']:
    # 타겟 열 선택
    train_target = train_data[[target_col]]

    # 데이터 증강
    new_train, new_target = gan_gen.generate_data_pipe(train_features, train_target, test_features)

    # 증강된 데이터 저장
    new_trains.append(new_train)
    new_targets.append(new_target)

# 증강된 데이터 확인
for i, target_col in enumerate(['색상_L*', '색상_a*', '색상_b*']):
    print(f"New train data for {target_col}:")
    print(new_trains[i].head())
    print(f"New target data for {target_col}:")
    print(new_targets[i].head())
    print()


Fitting CTGAN transformers for each column:   0%|          | 0/183 [00:00<?, ?it/s]

Training CTGAN, epochs::   0%|          | 0/500 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 2675, number of negative: 2675
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002018 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 519
[LightGBM] [Info] Number of data points in the train set: 5350, number of used features: 53
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positiv

Fitting CTGAN transformers for each column:   0%|          | 0/183 [00:00<?, ?it/s]

Training CTGAN, epochs::   0%|          | 0/500 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 2675, number of negative: 2675
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001962 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 519
[LightGBM] [Info] Number of data points in the train set: 5350, number of used features: 53
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positiv

Fitting CTGAN transformers for each column:   0%|          | 0/183 [00:00<?, ?it/s]

Training CTGAN, epochs::   0%|          | 0/500 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 2675, number of negative: 2675
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001808 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 519
[LightGBM] [Info] Number of data points in the train set: 5350, number of used features: 53
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positiv

In [ ]:
from tabgan.sampler import OriginalGenerator, GANGenerator, ForestDiffusionGenerator
import pandas as pd
import numpy as np

# random input data


# generate data
new_train1, new_target1 = OriginalGenerator().generate_data_pipe(train_features, train_targets, test_features, )
new_train2, new_target2 = GANGenerator().generate_data_pipe(train_features, train_targets, test_features, )
new_train3, new_target3 = ForestDiffusionGenerator().generate_data_pipe(train_features, train_targets, test_features,)

# example with all params defined
new_train4, new_target4 = GANGenerator(gen_x_times=1.1, cat_cols=None,
           bot_filter_quantile=0.001, top_filter_quantile=0.999, is_post_process=True,
           adversarial_model_params={
               "metrics": "AUC", "max_depth": 2, "max_bin": 100,
               "learning_rate": 0.02, "random_state": 42, "n_estimators": 100,
           }, pregeneration_frac=2, only_generated_data=False,
           gen_params = {"batch_size": 500, "patience": 25, "epochs" : 500,}).generate_data_pipe(train, target,
                                          test, deep_copy=True, only_adversarial=False, use_adversarial=True)

ValueError: ignored

In [ ]:
# 모든 타겟 데이터를 하나의 DataFrame으로 병합
merged_targets = pd.concat(new_targets, axis=1)

# 병합된 DataFrame을 CSV 파일로 저장
merged_targets.to_csv("/content/sample/combined_targets.csv", index=False)


OSError: ignored

In [ ]:
# 증강된 피처 데이터와 타겟 데이터 결합
final_train_data = pd.concat([new_train, merged_targets], axis=1)

# 최종 학습 데이터셋 확인
print(final_train_data.head())
final_train_data.to_csv("/content/sample/new_train2.csv")


NameError: ignored

In [ ]:
final_train_data.drop(["level_0"],axis=1,inplace=True)

NameError: ignored

In [ ]:
final_train_data.to_csv("/content/Untitled Folder/new_train2.csv")

NameError: ignored

In [ ]:
final_train_data

,소재_경사정보_V1,소재_경사정보_V2,소재_경사정보_V3,소재_경사정보_V4,소재_경사정보_V5,소재_경사정보_V6,소재_경사정보_V7,소재_경사정보_V8,소재_경사정보_V9,소재_경사정보_V10,...,배합_염료3_Black_prop,배합_염료1_Grey_prop,배합_염료2_Grey_prop,배합_염료3_Grey_prop,배합_염료2_Brown_prop,배합_염료1_Brown_prop,배합_염료3_Brown_prop,색상_L*,색상_a*,색상_b*
0,0,300,96,0,0,88,0,150,150,0,...,0,0,0,0,0,0,0,23.8100,59.3975,-21.990000
1,0,300,96,0,0,88,0,150,150,0,...,0,0,0,0,0,1,0,37.6600,59.5265,9.530000
2,0,300,96,0,0,88,0,150,150,0,...,0,0,0,0,0,1,0,53.4250,-2.2500,9.200000
3,0,300,96,0,0,88,0,150,150,0,...,0,0,0,0,0,0,0,33.4500,-2.6900,-20.550000
4,0,300,96,0,0,88,0,150,150,0,...,0,0,0,0,0,0,0,37.3200,-1.6400,-15.710000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3339,0,300,96,0,0,88,0,150,150,0,...,0,0,0,0,0,0,0,32.1140,61.7315,21.590000
3340,0,300,96,0,0,96,0,150,150,0,...,0,1,0,0,0,0,0,40.6050,62.7625,-1.209000
3341,0,300,96,0,0,96,0,150,150,0,...,0,1,0,0,0,0,0,72.9950,62.0850,-1.190500
3342,0,300,96,0,0,88,0,150,150,0,...,0,0,0,0,0,0,0,31.9625,62.4850,-17.668333


In [ ]:
test

,A,B,C,D
0,0.382732,-0.034242,1.096347,-0.234216
1,-0.347451,-0.581268,-1.632635,-1.567768
2,-1.179158,1.301428,0.895260,1.374964
3,-1.332212,-1.968625,-0.660056,0.175819
4,0.498690,1.047972,0.284280,1.742669
5,-0.222606,-0.913079,-1.681218,-0.888971
6,0.242118,-0.888720,0.936742,1.412328
7,-2.369587,0.864052,-2.239604,0.401499
8,1.224871,0.064856,-1.279689,-0.585431
9,-0.261645,-0.182245,-0.202897,-0.109883


In [ ]:
from tabgan.sampler import OriginalGenerator, GANGenerator, ForestDiffusionGenerator
import pandas as pd
import numpy as np

# random input data
train = pd.DataFrame(np.random.randint(-10, 150, size=(150, 4)), columns=list("ABCD"))
target = pd.DataFrame(np.random.randint(0, 2, size=(150, 1)), columns=list("Y"))
test = pd.DataFrame(np.random.randint(0, 100, size=(100, 4)), columns=list("ABCD"))

# generate data
new_train1, new_target1 = OriginalGenerator().generate_data_pipe(train, target, test, )
new_train2, new_target2 = GANGenerator().generate_data_pipe(train, target, test, )
new_train3, new_target3 = ForestDiffusionGenerator().generate_data_pipe(train, target, test, )

# example with all params defined
new_train4, new_target4 = GANGenerator(gen_x_times=1.1, cat_cols=None,
           bot_filter_quantile=0.001, top_filter_quantile=0.999, is_post_process=True,
           adversarial_model_params={
               "metrics": "AUC", "max_depth": 2, "max_bin": 100,
               "learning_rate": 0.02, "random_state": 42, "n_estimators": 100,
           }, pregeneration_frac=2, only_generated_data=False,
           gen_params = {"batch_size": 500, "patience": 25, "epochs" : 500,}).generate_data_pipe(train, target,
                                          test, deep_copy=True, only_adversarial=False, use_adversarial=True)

[LightGBM] [Info] Number of positive: 74, number of negative: 74
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000044 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 135
[LightGBM] [Info] Number of data points in the train set: 148, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with po

Fitting CTGAN transformers for each column:   0%|          | 0/5 [00:00<?, ?it/s]

Training CTGAN, epochs::   0%|          | 0/500 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 80, number of negative: 80
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000048 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 172
[LightGBM] [Info] Number of data points in the train set: 160, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with po

Fitting CTGAN transformers for each column:   0%|          | 0/5 [00:00<?, ?it/s]

Training CTGAN, epochs::   0%|          | 0/500 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 80, number of negative: 80
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000042 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 172
[LightGBM] [Info] Number of data points in the train set: 160, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with po

(      A   B   C   D
 0    55  54  11  51
 1    53  58   8  34
 2    54  49   6  46
 3    34  58   6  77
 4    49  46   3  27
 ..   ..  ..  ..  ..
 98    7  10  78  51
 99   37   3  56  12
 100  85  17  58  32
 101   0  16  43  67
 102   7  13  18  83
 
 [103 rows x 4 columns],
 0      0
 1      0
 2      0
 3      0
 4      1
       ..
 98     0
 99     0
 100    1
 101    0
 102    0
 Name: Y, Length: 103, dtype: int64)

In [ ]:
from tabgan.sampler import OriginalGenerator, GANGenerator
import pandas as pd
import numpy as np

# 원본 데이터셋 준비
train = pd.DataFrame(np.random.randint(10, 150, size=(50, 4)), columns=list("ABCD"))
target = pd.DataFrame(np.random.randint(0, 2, size=(50, 1)), columns=list("Y"))
test = pd.DataFrame(np.random.randint(0, 100, size=(100, 4)), columns=list("ABCD"))

# OriginalGenerator를 사용한 데이터 증강
new_train1, new_target1 = OriginalGenerator().generate_data_pipe(train, target, test)

# GANGenerator를 사용한 데이터 증강
new_train2, new_target2 = GANGenerator().generate_data_pipe(train, target, test)


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 13, number of negative: 12
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 25, number of used features: 0
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.520000 -> initscore=0.080043
[LightGBM] [Info] Start training from score 0.080043
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because the

Fitting CTGAN transformers for each column:   0%|          | 0/5 [00:00<?, ?it/s]

Training CTGAN, epochs::   0%|          | 0/500 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 30, number of negative: 29
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000052 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 76
[LightGBM] [Info] Number of data points in the train set: 59, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.508475 -> initscore=0.033902
[LightGBM] [Info] Start training from score 0.033902
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

In [ ]:
new_train2, new_target2

(     A   B   C   D
 0   30  76  76  88
 1   37  96  81  75
 2   79  82  65  55
 3   76  83  72  64
 4   87  72  99  56
 5   74  76  91  83
 6   27  91  41  75
 7   86  92  49  66
 8   14  64  89  46
 9   97  73   7  90
 10  93  71   3  65
 11  47   7  71  84
 12  50   5  72  90
 13  88  10  76  75
 14  70  15  63  70
 15  87  43  63  91
 16  95  70  34  97
 17  28  88   8  41
 18  40  77  75  14
 19   4  65  15  64
 20  26  86  82   4
 21  21  84  73  13
 22   8  16  43  69
 23  25  18  31  54
 24  16  46  51  74
 25  19  15  45  51
 26  29  42   7  55
 27  39  61  70  27
 28  97  90  49   6
 29  49  41  41  64
 30  22  59  98  24
 31  46  28  15  79
 32  77  75   6  30
 33  77  80  29  20
 34  16  40  91  16
 35  43  25  27  33
 36  57  34  41   9,
 0     0
 1     0
 2     0
 3     0
 4     0
 5     0
 6     1
 7     0
 8     0
 9     0
 10    1
 11    1
 12    0
 13    0
 14    1
 15    1
 16    0
 17    0
 18    1
 19    1
 20    0
 21    1
 22    0
 23    0
 24    0
 25    1
 26  

In [ ]:
# 가정: target이 연속형 데이터를 포함하고 있다고 가정
train = pd.DataFrame(np.random.uniform(10, 150, size=(50, 4)), columns=list("ABCD"))
target = pd.DataFrame(np.random.uniform(0, 1000, size=(50, 1)), columns=list("Y"))  # 연속형 데이터
test = pd.DataFrame(np.random.uniform(0, 100, size=(100, 4)), columns=list("ABCD"))

# 데이터 증강
new_train1, new_target1 = OriginalGenerator().generate_data_pipe(train, target, test)
new_train2, new_target2 = GANGenerator().generate_data_pipe(train, target, test)


[LightGBM] [Info] Number of positive: 26, number of negative: 25
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000042 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 64
[LightGBM] [Info] Number of data points in the train set: 51, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.509804 -> initscore=0.039221
[LightGBM] [Info] Start training from score 0.039221
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

Fitting CTGAN transformers for each column:   0%|          | 0/5 [00:00<?, ?it/s]

Training CTGAN, epochs::   0%|          | 0/500 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 25, number of negative: 24
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000033 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 72
[LightGBM] [Info] Number of data points in the train set: 49, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.510204 -> initscore=0.040822
[LightGBM] [Info] Start training from score 0.040822
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

### 결합추출


In [ ]:
from tabgan.sampler import GANGenerator

# 각 타겟 열에 대해 별도의 데이터 증강 수행 후, 타겟 데이터만 결합
new_train = None
new_targets_combined = pd.DataFrame()

for target_col in ['색상_L*', '색상_a*', '색상_b*']:
    current_train_target = train_targets[[target_col]]
    generated_train, generated_target = GANGenerator().generate_data_pipe(train_features, current_train_target, test_features)

    # 첫 번째 반복에서 증강된 특성 데이터 저장
    if new_train is None:
        new_train = generated_train

    # 증강된 타겟 데이터 결합
    new_targets_combined = pd.concat([new_targets_combined, generated_target], axis=1)

# 증강된 데이터 확인
print("New train data:")
print(new_train.head())
print("New combined target data:")
print(new_targets_combined.head())


Fitting CTGAN transformers for each column:   0%|          | 0/171 [00:00<?, ?it/s]

Training CTGAN, epochs::   0%|          | 0/500 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 2675, number of negative: 2675
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002075 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 520
[LightGBM] [Info] Number of data points in the train set: 5350, number of used features: 53
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positiv

Fitting CTGAN transformers for each column:   0%|          | 0/171 [00:00<?, ?it/s]

Training CTGAN, epochs::   0%|          | 0/500 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 2675, number of negative: 2675
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001852 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 515
[LightGBM] [Info] Number of data points in the train set: 5350, number of used features: 53
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positiv

Fitting CTGAN transformers for each column:   0%|          | 0/171 [00:00<?, ?it/s]

Training CTGAN, epochs::   0%|          | 0/500 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 2675, number of negative: 2675
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001707 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 517
[LightGBM] [Info] Number of data points in the train set: 5350, number of used features: 53
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positiv

In [ ]:
new_train,new_targets_combined

(      소재_경사정보_V1  소재_경사정보_V2  소재_경사정보_V3  소재_경사정보_V4  소재_경사정보_V5  소재_경사정보_V6  \
 0              0         300          96           0           0          88   
 1              0         300          96           0           0          88   
 2              0         300          96           0           0          88   
 3              0         300          96           0           0          88   
 4              0         300          96           0           0          88   
 ...          ...         ...         ...         ...         ...         ...   
 3339           0         300          96           0           0          96   
 3340           0         300          96           0           0          88   
 3341           0         300          96           0           0          88   
 3342           0         300          96           0           0          96   
 3343           0         300          96           0           0          88   
 
       소재_경사정보_V7  소재_경사정보

In [ ]:
# 증강된 특성 데이터와 타겟 데이터를 결합
final_train_dataset = pd.concat([new_train, new_targets_combined], axis=1)

# 최종 훈련 데이터셋 확인
print("Final train dataset:")
print(final_train_dataset.head())


Final train dataset:
   소재_경사정보_V1  소재_경사정보_V2  소재_경사정보_V3  소재_경사정보_V4  소재_경사정보_V5  소재_경사정보_V6  \
0           0         300          96           0           0          88   
1           0         300          96           0           0          88   
2           0         300          96           0           0          88   
3           0         300          96           0           0          88   
4           0         300          96           0           0          88   

   소재_경사정보_V7  소재_경사정보_V8  소재_경사정보_V9  소재_경사정보_V10  ...  배합_염료3_Black_prop  \
0           0         150         150            0  ...                  0   
1           0         150         150            0  ...                  0   
2           0         150         150            0  ...                  0   
3           0         150         150            0  ...                  0   
4           0         150         150            0  ...                  0   

   배합_염료1_Grey_prop  배합_염료2_Grey_prop  배합_염료3_G

In [ ]:
final_train_dataset.to_csv("/content/Untitled Folder/new_train3.csv")

In [ ]:
new_train, new_target = GANGenerator().generate_data_pipe(train_features, train_targets, test_features)


# example with all params defined
new_train4, new_target4 = GANGenerator(gen_x_times=1.1, cat_cols=None,
           bot_filter_quantile=0.001, top_filter_quantile=0.999, is_post_process=True,
           adversarial_model_params={
               "metrics": "AUC", "max_depth": 2, "max_bin": 100,
               "learning_rate": 0.02, "random_state": 42, "n_estimators": 100,
           }, pregeneration_frac=2, only_generated_data=False,
           gen_params = {"batch_size": 500, "patience": 25, "epochs" : 500,}).generate_data_pipe(train, target,
                                          test, deep_copy=True, only_adversarial=False, use_adversarial=True)

ValueError: ignored